In [1]:
import pandas as pd
import numpy as np
import os
from dateutil.relativedelta import relativedelta
from datetime import date, datetime
import xgboost as xgb
import pickle
import cx_Oracle
os.chdir(r"C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\Supplier_Risk")

In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
import time
import pickle
import os
from math import log
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, ParameterGrid,StratifiedKFold, cross_val_score

from sklearn import metrics 
from sklearn.metrics import confusion_matrix, precision_recall_curve, average_precision_score, brier_score_loss, f1_score, log_loss,auc,roc_curve,auc

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.decomposition import PCA
#from sklearn.utils.fixes import signature

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
os.chdir(r"C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\Supplier_Risk")

### Import Data Files

In [3]:
tns_dsn = cx_Oracle.makedsn('aamlxbidbp001.aam.net', 1525, service_name='BIDWPRD')
con_ORACLE = cx_Oracle.connect(user='PROD_AAM_RO', password='prodaamro', dsn=tns_dsn)

df = pd.read_sql_query("""select * from bi_dw.WC_SC_SUMMARY_V
                       where month >= 202101
                       """, con_ORACLE)

In [4]:
df_vb = pd.read_sql_query("""select b.supplier_name as vendor_name, third_party_id,  vulnerability_score
                            from 
                            (select third_party_id , max(score) vulnerability_score from rpa_kpmg_financial
                            group by third_party_id
                            ) a
                            join 
                            bi_dw.wc_sc_supplier_d b 
                            on a.third_party_id = b.affiliate_code 
                       """, con_ORACLE)

In [5]:
df_vb.dropna(subset = ["VULNERABILITY_SCORE"], inplace = True)

In [6]:
df_vb["VULNERABILITY_SCORE"] = df_vb["VULNERABILITY_SCORE"].astype(float)

In [7]:
# Download data from Orcale DB for prod
# Logic to collate data and filter suppliers to be implemented in the SQL script itself
# df = pd.read_csv(os.getcwd()+r"\Data Files\Feature Engineering Base.csv", encoding='latin-1')
df1 = pd.read_csv(os.getcwd()+r"\data\Mapping File Prod.csv", encoding='latin-1')

In [8]:
df = df.merge(df1, on = ["VENDOR_NAME","ORGANIZATION_CODE"], how='left')

In [9]:
df = df[df["ORG_TYPE"]!="Others2"]
df = df.groupby(['VENDOR_NAME', 'MONTH', 'TYPE']).sum().reset_index()

In [10]:
df = df.pivot(index = ['VENDOR_NAME','MONTH'], columns=['TYPE'], values = 'VAL').reset_index()

### Data Cleaning 

In [11]:
df["Qty Recd"].fillna(0, inplace = True)

In [12]:
df["QTY_RECD_FLAG"] = np.where((df["Qty Ordered"] != np.NaN) & (df["Qty Ordered"] !=0), df["Qty Recd"]/df["Qty Ordered"], np.NaN)
df["QTY_RECD_FLAG"] = np.where(df["QTY_RECD_FLAG"]>1, 1, df["QTY_RECD_FLAG"])

In [13]:
df["PPM Flag"] = np.where((df["PPM Qty Shipped"] != np.NaN) & (df["PPM Qty Shipped"] != 0), df["PPM Qty Rejected"]/df["PPM Qty Shipped"]*1000000, np.NaN)

In [14]:
df["MONTH"] = pd.to_datetime(df["MONTH"], format = '%Y%m').dt.date

### Define Current Month

In [15]:
cur_month = date(date.today().year, date.today().month, 1)

# cur_month = date(2022,4,1)

In [16]:
df = df[df["MONTH"]>= cur_month - relativedelta(months=12)]

### QTY_RECD_FLAG to be aggregated w/o vendor name

In [17]:
target_df = df[["VENDOR_NAME"]]
target_df.drop_duplicates(inplace = True)

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
final_target_df = pd.DataFrame()


month = cur_month

target_df = df[["VENDOR_NAME"]]
target_df.drop_duplicates(inplace = True)


#Month on Month features
for z in np.arange(1,13,1):
    tmp = df[df["MONTH"]==month - relativedelta(months=z)].groupby(['VENDOR_NAME']).agg({ 
                "QTY_RECD_FLAG" : ['mean'],
                "Qty Ordered":['sum'], 
                "Qty Recd":['sum'],
                "PRNR" : ['sum'],
                "PREM FREIGHT" : ['sum'],
                "Cost Recovery" : ['sum'],
                "PPM Qty Rejected": ['sum'],
                "PPM Qty Shipped":['sum']
                })
    tmp.columns = [s + "_MOM_"  + str(z) for s in ["_".join(x) for x in tmp.columns.ravel()]]
    tmp.reset_index(inplace = True)
    target_df = target_df.merge(tmp, how = 'left', on = 'VENDOR_NAME')


#Quarterly features
for z in np.arange(1,13,3):
    tmp = df[(df["MONTH"]>=month - relativedelta(months=z+2))&(df["MONTH"]<=month - relativedelta(months=z))].groupby(['VENDOR_NAME']).agg({ 
                "QTY_RECD_FLAG" : ['max','min','std','mean'],
                "Qty Ordered":['max','min','std','mean', 'sum'], 
                "Qty Recd":['max','min','std','mean','sum'],
                "PRNR" : ['max','std','mean','sum'],
                "PREM FREIGHT" : ['max','std','mean','sum'],
                "Cost Recovery" : ['max','std','mean','sum'],
                "PPM Flag" : ['max','min','std','mean'], 
                "PPM Qty Rejected": ['max','min','std','mean', 'sum'],
                "PPM Qty Shipped":['max','min','std','mean', 'sum']
                })
    tmp.columns = [s + "_OQT_"  + str(z) for s in ["_".join(x) for x in tmp.columns.ravel()]]
    tmp.reset_index(inplace = True)
    
    for z in [x for x in tmp.columns if "PPM Qty Shipped_sum" in x]:
        tmp[z.replace("PPM Qty Shipped_sum","PPM_Agg_Flag")] = np.where((tmp[z]!= np.NaN) & (tmp[z]!= 0), np.round(tmp[z.replace("Shipped","Rejected")]/tmp[z]*1000000), np.NaN)
        
    for z in [x for x in tmp.columns if "Qty Ordered_sum" in x]:
        tmp[z.replace("Qty Ordered_sum","Qty Recd_percent_Agg")] = np.where((tmp[z]!= np.NaN) & (tmp[z]!= 0), tmp[z.replace("Ordered","Recd")]/tmp[z]*100, np.NaN) 


    target_df = target_df.merge(tmp, how = 'left', on = 'VENDOR_NAME')


final_target_df = pd.concat([final_target_df, target_df])

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


# Merge with Vulnerability DF to map target score

In [19]:
final_target_df = final_target_df.merge(df_vb, how = 'inner', on = 'VENDOR_NAME')

In [20]:
# selected_vars_vulnerability = ['Qty Recd_sum_OQT_7', 'Qty Recd_mean_OQT_7', 'PRNR_sum_MOM_7','Qty Ordered_sum_OQT_7', 
#                                'Qty Ordered_sum_MOM_8','PPM Qty Shipped_std_OQT_4', 'QTY_RECD_FLAG_mean_MOM_2',
#        'QTY_RECD_FLAG_mean_OQT_4', 'Qty Recd_min_OQT_4', 'Qty Recd_max_OQT_7','PPM Qty Shipped_std_OQT_1', 'PPM Qty Shipped_sum_MOM_5',
#        'PPM Flag_mean_OQT_1', 'QTY_RECD_FLAG_mean_MOM_10','QTY_RECD_FLAG_mean_MOM_4', 'QTY_RECD_FLAG_mean_OQT_1',
#        'Qty Ordered_mean_OQT_7', 'Cost Recovery_mean_OQT_1','PPM Qty Shipped_sum_MOM_8']

In [21]:
selected_vars_vulnerability = ['PPM Flag_std_OQT_1',
  'PPM Qty Shipped_min_OQT_10',
  'PPM Flag_mean_OQT_1',
  'PPM Qty Shipped_sum_MOM_5',
  'PPM Qty Shipped_sum_OQT_4',
  'PRNR_std_OQT_10',
  'PPM Qty Shipped_max_OQT_4',
  'PPM Qty Shipped_std_OQT_1',
  'Qty Recd_sum_MOM_4',
  'Qty Ordered_std_OQT_4',
  'Qty Recd_percent_Agg_OQT_1',
  'Qty Ordered_std_OQT_10',
  'PPM Flag_max_OQT_1',
  'Qty Recd_min_OQT_4',
  'Qty Recd_mean_OQT_7',
  'QTY_RECD_FLAG_mean_MOM_12',
  'Qty Recd_std_OQT_7',
  'PPM Qty Shipped_std_OQT_7',
  'Qty Ordered_sum_OQT_7']

In [22]:
bst = pickle.load(open("xgb_vulnerability_reg", "rb"))

In [23]:
final_target_df["PRED_VULNERABILITY_SCORE"] =  bst.predict(final_target_df[selected_vars_vulnerability], ntree_limit=bst.best_ntree_limit)

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\xgboost\core.py:108: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


In [24]:
weight = 0.2

In [25]:
final_target_df["PRED_VULNERABILITY_SCORE_FINAL"] = ((1-weight)*final_target_df["VULNERABILITY_SCORE"])+(weight * final_target_df["PRED_VULNERABILITY_SCORE"])

In [26]:
final_target_df["PRED_VULNERABILITY"] = final_target_df["PRED_VULNERABILITY_SCORE_FINAL"].round(5)

In [27]:
final_target_df["FLAG_VULNERABILITY"] =  np.where(final_target_df["PRED_VULNERABILITY_SCORE_FINAL"]>=55,3,np.where(final_target_df["PRED_VULNERABILITY_SCORE_FINAL"]>=35,2,np.where(final_target_df["PRED_VULNERABILITY_SCORE_FINAL"]>=10,1,0)))

In [28]:
final_target_df["MONTH"] = cur_month

In [29]:
final_target_df

,VENDOR_NAME,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,Cost Recovery_sum_MOM_1,PPM Qty Rejected_sum_MOM_1,PPM Qty Shipped_sum_MOM_1,QTY_RECD_FLAG_mean_MOM_2,...,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10,THIRD_PARTY_ID,VULNERABILITY_SCORE,PRED_VULNERABILITY_SCORE,PRED_VULNERABILITY_SCORE_FINAL,PRED_VULNERABILITY,FLAG_VULNERABILITY,MONTH
0,AALBERTS T S/ IMPREGLON,1.000000,1605.0,1605.0,0.0,0.0,0.0,0.0,1605.0,1.000000,...,2886.0,0.0,100.000000,1022,25.62,26.304279,25.756856,25.75686,1,2023-02-01
1,AAM AUTO COMPONENT (INDIA) PRIVATE LIMITED,0.343099,26386.0,9053.0,0.0,0.0,0.0,0.0,35728.0,0.136105,...,7271.0,0.0,351.632731,1,53.47,36.636005,50.103201,50.10320,2,2023-02-01
2,AAM CHANGSHU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-32892.0,-0.0,NaN,1,53.47,39.401672,50.656334,50.65633,2,2023-02-01
3,AAM DO BRASIL LTDA,0.000000,720.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,49582.0,0.0,92.446201,1,53.47,37.587837,50.293568,50.29357,2,2023-02-01
4,AAM GERMANY GMBH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,1,53.47,48.297245,52.435450,52.43545,2,2023-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,ZHEJIANG SHUANGHUAN DRIVELINE CO LTD,0.979866,10728.0,10512.0,4.0,0.0,0.0,24.0,15494.0,0.921097,...,133755.0,135.0,109.395213,1026,24.15,39.720200,27.264040,27.26404,1,2023-02-01
571,ZHONGDING USA INC,0.707597,23601.0,16700.0,0.0,0.0,0.0,0.0,16700.0,1.000000,...,216200.0,0.0,100.000000,1192,29.80,37.916039,31.423208,31.42321,1,2023-02-01
572,ZOUPING TIANSHENG METAL TECHNOLOGY CO LTD,1.000000,17022.0,17766.0,0.0,0.0,0.0,0.0,16542.0,0.888879,...,75832.0,0.0,93.629000,82,38.05,38.606323,38.161265,38.16126,2,2023-02-01
573,ZWZ BEARING USA,1.000000,199584.0,219888.0,0.0,0.0,0.0,0.0,211248.0,0.958230,...,563438.0,0.0,100.426012,75,95.00,39.705933,83.941186,83.94119,3,2023-02-01


# Output 

### Save Output to CSV 

In [30]:
final_target_df[["VENDOR_NAME","MONTH","PRED_VULNERABILITY", "FLAG_VULNERABILITY"]].to_csv(os.getcwd()+r"\data\output_file_prod_vulnerability.csv", index=False)

### Delete older entries from SQL

In [31]:
sql = ("delete from bi_dw.wc_sc_pred_s_vul where MONTH = TO_DATE('"+str(cur_month)+"', 'YYYY-MM-DD' )")

cursor = con_ORACLE.cursor()

cursor.execute(sql)

con_ORACLE.commit()
cursor.close
con_ORACLE.close  

<function Connection.close>

### Save Output to Oracle

In [32]:
sql = ('insert into bi_dw.wc_sc_pred_s_vul(VENDOR_NAME,MONTH,PRED_VULNERABILITY,FLAG_VULNERABILITY)'
        'values(:VENDOR_NAME,:MONTH,:PRED_VULNERABILITY,:FLAG_VULNERABILITY)')

cursor = con_ORACLE.cursor()

upload_df = final_target_df[["VENDOR_NAME","MONTH","PRED_VULNERABILITY", "FLAG_VULNERABILITY"]]
upload_df.dropna(subset = ["VENDOR_NAME"], inplace = True)

df_list = upload_df.values.tolist()

cursor.executemany(sql, df_list)

con_ORACLE.commit()
cursor.close
con_ORACLE.close       

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


<function Connection.close>

# Check Database 

In [33]:
backup_v1=pd.read_sql_query("select * from bi_dw.wc_sc_pred_s_vul order by month desc", con_ORACLE)
# backup.to_csv(os.getcwd()+r"\Data Files\backup_prod.csv", index=False)
backup_v1.head()

,VENDOR_NAME,MONTH,PRED_VULNERABILITY,FLAG_VULNERABILITY
0,"NEK CO., LTD.",2023-02-01,27.32787,1
1,NELCAST LIMITED,2023-02-01,60.33757,3
2,NEMAK MEXICO SA,2023-02-01,31.22183,1
3,NEMAK USA INC,2023-02-01,32.93725,1
4,NEOSYM INDUSTRY LTD,2023-02-01,33.09897,1


In [34]:
backup_v1.MONTH.value_counts()

2022-11-01    619
2022-10-01    588
2022-12-01    584
2023-01-01    584
2022-07-01    578
2023-02-01    575
2022-09-01    563
2022-08-01    558
2022-01-01    402
2021-07-01    402
2021-08-01    402
2021-09-01    402
2022-03-01    402
2021-12-01    402
2022-04-01    402
2021-10-01    402
2021-11-01    402
2021-06-01    402
2022-02-01    402
2022-05-01    401
2022-06-01    401
Name: MONTH, dtype: int64

In [ ]:
backup_v1.to_csv(os.getcwd()+r"\data\validate_vul.csv", index=False)

In [ ]:
con_ORACLE.commit()
cursor.close
con_ORACLE.close    